In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

0) Kurulum & İçeri Aktarım

In [3]:
import os, pathlib, numpy as np, matplotlib.pyplot as plt, warnings
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, classification_report

2025-09-24 08:18:28.308746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758701908.708458      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758701908.809007      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})

1) Sabitler & Yol

In [12]:
SEED = 58
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
DATA_DIR = "/kaggle/input/combined-forest-fire/combined_data/train"
TEST_DIR = "/kaggle/input/combined-forest-fire/combined_data/test"
VAL_SPLIT = 0.2
AUTOTUNE  = tf.data.AUTOTUNE

tf.random.set_seed(SEED)
np.random.seed(SEED)

2) Dataset Oluşturma (train/val otomatik böl)

In [13]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import experimental as tfdata_exp

train_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="rgb"
)
val_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="rgb"
)

class_names = train_ds.class_names

test_ds = image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode="rgb",
    shuffle=False,               
    class_names=class_names        
)

NUM_CLASSES = len(class_names)
print("classes:", class_names)

train_ds = train_ds.apply(tfdata_exp.ignore_errors())
val_ds = val_ds.apply(tfdata_exp.ignore_errors())
test_ds = test_ds.apply(tfdata_exp.ignore_errors())

Found 20620 files belonging to 2 classes.
Using 16496 files for training.
Found 20620 files belonging to 2 classes.
Using 4124 files for validation.
Found 2935 files belonging to 2 classes.
classes: ['fire', 'nonfire']


3) Performans için önbellek & veri artırma (augmentation)

In [14]:
from tensorflow import keras
from tensorflow.keras import layers
# performans
train_ds = train_ds.shuffle(1024).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)

# augmentation katmanı (GPU'da çalışır)
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="augmentation")

4) Basit CNN– “baseline”


In [15]:
def build_baseline(input_shape=IMG_SIZE+(3,), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)

    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="cnn_baseline")
    return model

model = build_baseline()
model.summary()

Model: "cnn_baseline"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augmentation (Sequential)       │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 40, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,647,234 (25.36 MB)

 Trainable params: 6,647,234 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

5) Derleme, Callback’ler, Eğitim

In [17]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(train_ds, validation_data=val_ds, epochs=15,callbacks=callbacks)

model.evaluate(test_ds, verbose=1)

Epoch 1/15


Invalid SOS parameters for sequential JPEG


    278/Unknown 122s 41ms/step - accuracy: 0.8424 - loss: 0.4026

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
Invalid SOS parameters for sequential JPEG


278/278 ━━━━━━━━━━━━━━━━━━━━ 154s 156ms/step - accuracy: 0.8426 - loss: 0.4021 - val_accuracy: 0.9199 - val_loss: 0.1995 - learning_rate: 0.0010
Epoch 2/15


...

Invalid SOS parameters for sequential JPEG


278/278 ━━━━━━━━━━━━━━━━━━━━ 139s 133ms/step - accuracy: 0.9638 - loss: 0.0867 - val_accuracy: 0.9563 - val_loss: 0.1366 - learning_rate: 2.5000e-04
Epoch 15/15


Invalid SOS parameters for sequential JPEG


278/278 ━━━━━━━━━━━━━━━━━━━━ 139s 131ms/step - accuracy: 0.9660 - loss: 0.0829 - val_accuracy: 0.9506 - val_loss: 0.1402 - learning_rate: 2.5000e-04
78/78 ━━━━━━━━━━━━━━━━━━━━ 38s 472ms/step - accuracy: 0.9013 - loss: 0.2632


[0.23083461821079254, 0.9135504364967346]

In [21]:
model.save("/kaggle/working/my_model.h5")
#model.save_weights("/kaggle/working/my_model.weights.h5")

Tek görseli tahmin et (top-1 olasılıkla)

In [18]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np, pathlib

# IMG_SIZE ve class_names önceki hücrelerden geliyor
def predict_image(img_path, model, img_size=IMG_SIZE, class_names=class_names):
    # Güvenli RGB + yeniden boyutlandırma
    img = load_img(img_path, target_size=img_size, color_mode="rgb")
    arr = img_to_array(img)              # (H,W,3), dtype=float32, 0..255
    arr = np.expand_dims(arr, axis=0)    # (1,H,W,3)

    # Tahmin
    probs = model.predict(arr, verbose=0)[0]          # (num_classes,)
    idx   = int(np.argmax(probs))
    pred  = class_names[idx]
    conf  = float(probs[idx])

    return {"path": str(img_path), "pred": pred, "conf": conf, "probs": probs}

# KULLANIM:
# 1) Kaggle'da "Add data" → kendi görsellerini bir dataset olarak ekle veya
# 2) /kaggle/working içine upload et (Notebook arayüzünden).
TEST_IMG = "/kaggle/input/testtest/testdata/nonfiretest/NF_9.jpg"  # <-- kendi dosya yolunu yaz
res = predict_image(TEST_IMG, model)
print(f"Pred: {res['pred']}  |  conf: {res['conf']:.3f}")

Pred: nonfire  |  conf: 0.984


In [ ]:
model.save_weights("/kaggle/working/fire_nonfire.weights.h5")

In [19]:
import glob, os
from PIL import Image, UnidentifiedImageError

def predict_folder(folder_path, model, img_size=IMG_SIZE, class_names=class_names):
    exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(folder_path, e)))
    results, bad = [], []

    for p in files:
        try:
            # Önce açılıp RGB'ye çevrilebildiğini hızlıca kontrol edelim (bozukları atla)
            with Image.open(p) as im:
                im.convert("RGB")
            out = predict_image(p, model, img_size, class_names)
            results.append(out)
        except (UnidentifiedImageError, OSError):
            bad.append(p)

    # küçük bir özet tablosu
    import pandas as pd
    if results:
        df = pd.DataFrame([{"path": r["path"], "pred": r["pred"], "conf": r["conf"]} for r in results])
        display(df.sort_values("conf", ascending=False).reset_index(drop=True))
        print(f"\nToplam: {len(results)} görüntü tahmin edildi. Atlanan (bozuk): {len(bad)}")
    else:
        print("Hiç uygun görüntü bulunamadı.")
    if bad:
        print("Atlanan (bozuk) örnekler (ilk 10):")
        for p in bad[:10]:
            print(" -", p)

# KULLANIM:
TEST_DIR = "/kaggle/input/testtest/testdata/firetest"  # <-- klasör yolunu yaz (içinde .jpg/.png dosyaların olsun)
predict_folder(TEST_DIR, model)

,path,pred,conf
0,/kaggle/input/testtest/testdata/firetest/F_9.jpg,fire,1.000000
1,/kaggle/input/testtest/testdata/firetest/F_6.jpg,fire,0.999459
2,/kaggle/input/testtest/testdata/firetest/F_5.jpg,fire,0.997867
3,/kaggle/input/testtest/testdata/firetest/F_8.jpg,fire,0.995633
4,/kaggle/input/testtest/testdata/firetest/F_10.jpg,fire,0.986421
5,/kaggle/input/testtest/testdata/firetest/F_11.jpg,fire,0.970741
6,/kaggle/input/testtest/testdata/firetest/F_7.jpg,fire,0.967110



Toplam: 7 görüntü tahmin edildi. Atlanan (bozuk): 0
